In [62]:
import pandas as pd
from collections import defaultdict
import numpy as np

df = pd.read_csv(r"Real_Data\primaryschool.csv", sep="\t", header=None)

merged = defaultdict(set)

for node_col, attr_col in [(1, 3), (2, 4)]:
    for node, attrs in df.groupby(node_col)[attr_col]:
        merged[node].update(attrs)

meta_df = (
    pd.DataFrame(
        [(node, next(iter(attrs))) for node, attrs in merged.items()],
        columns=["node", "attribute"],
    )
    .sort_values("attribute")
    .reset_index(drop=True)
)

node_mapping = dict(zip(meta_df["node"], meta_df.index))
node_to_attribute = dict(zip(meta_df.index, meta_df["attribute"]))

df = df[[0, 1, 2]]
df.columns = ["starting_times", "source_nodes", "target_nodes"]

df["source_nodes"] = df["source_nodes"].map(node_mapping)
df["target_nodes"] = df["target_nodes"].map(node_mapping)


MIN_TIME = 8.5 * 3600
df["starting_times"] = df["starting_times"] - MIN_TIME
df = df[df["starting_times"] > 24 * 3600].reset_index(drop=True)
df["starting_times"] = df["starting_times"] - 24 * 3600
df['starting_times']=df['starting_times']-df['starting_times'].min()
df["ending_times"] = df["starting_times"] + 20
df['durations']=df["ending_times"] -df["starting_times"] 
df['source_nodes']=df['source_nodes'].astype(int)
df['target_nodes']=df['target_nodes'].astype(int)

df['ending_times']=df['ending_times'].astype(int)
df['starting_times']=df['starting_times'].astype(int)
df['durations']=df['durations'].astype(int)

df['source_nodes']=df['source_nodes'].map(mapping)
df['target_nodes']=df['target_nodes'].map(mapping)

df.to_csv('Pre_processed_Data\primary_school_day2.csv', index=False)

In [63]:
missing = {121, 172, 207, 213}
all_nodes = list(range(242))  # 1 to 242 inclusive
present_nodes = [n for n in all_nodes if n not in missing]

# Map each present node to a new continuous label starting from 0
mapping = {old: new for new, old in enumerate(present_nodes)}


meta_df.reset_index(names='node_name', inplace=True)
class_mapping={mapping.get(meta_df.iloc[i]['node_name'], 'unknown'):meta_df.iloc[i]['attribute'] for i in range(len(meta_df))}
del class_mapping['unknown']
np.save('Pre_processed_Data/mapping_dict.npy', class_mapping)